In [1]:
using DataFrames, CSV

In [2]:
rawdatadir = joinpath("..","raw")
demanddata = Dict()
vredata = Dict()
readdir(rawdatadir, join=true)


7-element Vector{String}:
 "../raw/demanddata_2015.csv"
 "../raw/demanddata_2016.csv"
 "../raw/demanddata_2017.csv"
 "../raw/demanddata_2018.csv"
 "../raw/demanddata_2019.csv"
 "../raw/ninja_pv_country_GB_merra-2_corrected.csv"
 "../raw/ninja_wind_country_GB_current-merra-2_corrected.csv"

In [3]:
years = []
yearlydemanddata = Dict()
separatedvredata = Dict()
for file in readdir(rawdatadir, join=true)
    if occursin("demanddata_", file)
        y = parse(Int64,(split(split(file,"_")[2],".")[1]))
        push!(years,y)
        yearlydemanddata[y] = select!(CSV.read(file,DataFrame),1:2,4)
    elseif occursin("wind", file)
        separatedvredata["wind"] = CSV.read(file, DataFrame, header=3)
    elseif occursin("pv", file)
        separatedvredata["pv"] = CSV.read(file, DataFrame, header=3)
    else
        println("Fail to read ", file)
    end
end
start_year = minimum(years)
end_year = maximum(years);

In [4]:
demanddata = DataFrame()
for y in start_year:end_year
    append!(demanddata,yearlydemanddata[y])
end
rename!(demanddata, :TSD => :load)
demanddata[!,:hh] = 1:nrow(demanddata)
select!(demanddata,"hh","load")
demanddata;


In [5]:
pv = DataFrame()
wind = DataFrame()
for y in start_year:end_year
    append!(pv, separatedvredata["pv"][map(x->occursin(string(y),x),separatedvredata["pv"][:,"time"]),:])
    append!(wind, separatedvredata["wind"][map(x->occursin(string(y),x),separatedvredata["wind"][:,"time"]),:])
end
pv = pv[:,[:national]]
rename!(pv, :national=>:cf)
pv[!,:vre_id] .= 1
onshore = wind[:,[:onshore]]
rename!(onshore, :onshore=>:cf)
onshore[!,:vre_id] .= 2
offshore = wind[:,[:offshore]]
rename!(offshore, :offshore=>:cf)
offshore[!,:vre_id] .= 3

function hourytohalfhourly(df)
    df = repeat(df, inner=2)
    df[!,:hh] = 1:nrow(df)
    select!(df, :hh, :vre_id, :cf)
end

pv = hourytohalfhourly(pv)
onshore = hourytohalfhourly(onshore)
offshore = hourytohalfhourly(offshore)
vredata = [pv;onshore;offshore];

In [6]:
CSV.write(joinpath("..","cleaned","gbhalfhourlydemand.csv"),demanddata)
CSV.write(joinpath("..","cleaned","gbhalfhourlyvrecf.csv"),vredata);